In [2]:
from Bio import SeqIO


seq = None
for seq_record in SeqIO.parse("files/CP011547.gbk", "genbank"):
    # print(f"ID: {seq_record.id}")
    # print(repr(seq_record.seq))
    # print(f"Number of nucleotides: {len(seq_record)}")

    seq_record.translate()
    translation = seq_record.translate()
    
    seq = seq_record
    
    # print(seq_record)
    # print("=========================================")
    # print(translation)

    
    # SeqIO.write(seq_record, f"{seq_record.id}.fasta", "fasta")
     



In [22]:
reading_frames = []
for amino_acid in SeqIO.parse("works.fas", "fasta"):
    reading_frames.append(amino_acid)

In [23]:
reading_frames

[SeqRecord(seq=Seq(''), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('HTTPTHPHTTPHTTPHPHTHILTLP*HSPNLTLANLSLNLPSITLPPLVTLSHS...VCV'), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('TPHPHTHTPHHTPHHTHTHTS*HYPNTALI*PWPTCLSTYPPLPCLHSLPCPIQ...CVW'), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq(''), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('GVCGVGVVWVCGVVWVWVWVWVWVWVWCGCVVWDYECWL*NCPYNIW*HVYTLF...HTT'), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('VCVVWVWCGCVVWCGCGCGCGCGCGCGVGVWCGITNVGYRTVLTTFGNMFIPYL...TPH'), id='<unknown', name='<unknown', description='<unknown id> <unknown description>', dbxrefs=[])]

In [19]:
translation = seq.seq.translate()

print(translation)

PHHTHTPTHHTTHHTTPTHTHPNTTLTQP*SNPGQPVSQLTLHYPASTRYPVPFNHTTPNHHPSLYLLPLTHRYPPITHIQPTATYPTITLPSTMTYSPYCSSTHHIETLTNDRK*HTRAYPTTLYHHHMPYSPSLVY*FYVRTRMLQYIPSQTYPTLRFHFTPWPISH*ISTKCTHIIMHGTCLSGLYPVPFTHNAHHYPHFDIYISFGGPKYCITALNTYVIPLLHHILTTPFIYTYVNITEKSPQKSPKHKNILLFNNNT*TYWLVVATLSWYH*RKSSSILQFA*TDAISEYFVLTQAIH*NNMSHHCRNTLYSPSNNTVVAQTHAGAMIQLYLISIPIC*PQYPKSITDASLILYVTLLIRRDYI*SRRYCDRYVI**DL*RNVK*FYGNITFHRRILKRMLRY*LTLSYHPLSYC**NTNPSALFLVTFTQKTMPTQKS*YFTCQKMRVSK*EFGTMTCNSHCPDLQSCS*K*RIFYTARRDAPKNEKRSSDSFLFKDKGCEAAHFQFHCCLLDIHC*LYYRPRFFYNSVEVSFLCSSYS*NASRTPSLIK*VYNINIHLYNLRYLYHQKKVVFLFYFVR*FSISMETRS*NWRLSLVCDSVDTVLG*STGDGWL*SAGVPWNTGDHSGHLVWSNTGQHGGEVTVVENGFSNFDWVGFSWVGGLEHVVLG*VSSDIRDVDTQFHQVDSFVRLS*SGGCRSSSSDGSDTSGD*S*FDHCICFVC*C*YKLNRKGKNKDIFSKAYS*SSSIYTHSLMGCCYLNDR*LAPVPHQIFSISHLSHNLIISMEMLLFLNES*IFHRFRMWSTVLWRLCVFVCAECGNANYRGAEVPYKTLFCACDISFFGQKEYPNFRFGPSYRSLLSKPEFSLL*TASAEEIFPSLEFVQH*TCVGSRILLGSVNL*TLGKCLGAIT*F*PLRSGW**DKLISNRYIFKRGYR*FSRAVLL*FDMYG*LNLSRDMRVRTFGYSSF*VGFFLILGFLKGY*SSAQRTLGN

In [16]:
for i in range(2):
    print(i)

0
1


In [38]:
seq_record = SeqIO.parse("files/CP011547.gbk", "genbank")

print(seq_record)

# print(f"ID: {seq_record.id}")
# print(repr(seq_record.seq))
# print(f"Number of nucleotides: {len(seq_record)}")

# print()
# print(seq_record.translate())
     